In [11]:
import requests
from bs4 import BeautifulSoup
import re
import pandas as pd

In [12]:
PATTERN_HEADLINE_TEXT = re.compile(r"(?<=\"headline\">)[A-Öa-ö 0-9.!?,-]+(?=<\/h1>)")
PATTERN_BROAD_TEXT = re.compile(r"(?<=css-10r2ygq\">)[A-Öa-ö 0-9.!?,-]+(?=<)")
PATTERN_LINK_TEXT = re.compile(r"(?<=>)[A-Öa-ö 0-9.!?,-]+(?=<\/a>)")
ARTICLE_DATE = re.compile(r"(?<=Publicerad:)[A-Öa-ö 0-9.!?,-]+(?=\")")

In [13]:
def collect_data(source_page):
    title = re.findall(PATTERN_HEADLINE_TEXT, source_page)
    date = re.findall(ARTICLE_DATE, source_page)
    collected = []
    for match in re.finditer(PATTERN_BROAD_TEXT, source_page):
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    if not collected:
        #return ["NULL"], "NULL", "NULL"
        return "NULL","NULL","NULL"
    if not title:
        title = ["NULL"]
    if not date:
        date = ["NULL"]

    # Find span for broad text
    first_broad_span = collected[0][0][0]
    last_broad_span = collected[-1][0][1]

    # correct span for broad texts now when broadtext chunk span is found
    collected = [((n[0][0] - first_broad_span, n[0][1] - first_broad_span), n[1]) for n in collected]

    for match in re.finditer(PATTERN_LINK_TEXT, source_page[first_broad_span:last_broad_span]):
        text = match.group() + " "
        span = match.span()
        data = (span, text)
        collected.append(data)

    collected.sort(key=lambda s: s[0])
    text_final = ""
    for data in collected:
        text_final += data[1]

    if not title:
        title = ["NULL"]

    return title[0], text_final, date[0]

In [39]:
df = pd.read_csv("links_mord, göteborg.csv", sep=",")
links =  df["link"]


0       https://www.aftonbladet.se/nyheter/a/GGkbjq/an...
1       https://www.aftonbladet.se/nyheter/a/Qm5XXx/mi...
2       https://www.aftonbladet.se/nyheter/a/Joyb46/mi...
3       https://www.aftonbladet.se/nyheter/a/Al20Jx/ma...
4       https://www.aftonbladet.se/nyheter/a/rAkno3/at...
                              ...                        
3287    https://www.aftonbladet.se/nyheter/a/gP0Q2a/sk...
3288    https://www.aftonbladet.se/nojesbladet/tv/a/4d...
3289    https://www.aftonbladet.se/nojesbladet/a/RxyB1...
3290    https://www.aftonbladet.se/kultur/a/L01BX9/sty...
3291    https://www.aftonbladet.se/nojesbladet/tv/a/6n...
Name: link, Length: 3292, dtype: object

In [19]:
from tqdm import tqdm

dates=[]
titles=[]
texts=[]

  
for url in tqdm(links, desc="Loading..."):
#for url in links:
    response = requests.get(url)
    soup = BeautifulSoup(response.content)
    #print(url)
    #print(soup)
    title, text, date = collect_data(str(soup))
    dates.append(date)
    titles.append(title)
    texts.append(text)

df2 = pd.DataFrame()
df2["date"] = dates  
df2["title"] = titles
df2["text"] = texts

Loading...: 100%|██████████| 3292/3292 [16:19<00:00,  3.36it/s]


In [20]:
df2.head()

,date,title,text
0,28 maj 2020 kl. 12.07,Anhållen efter misstänkt mord i Göteborg,Polisen misstänker att en man kan ha blivit mö...
1,07 januari 2021 kl. 08.35,Misstänkt mord i Göteborg,En man misstänks ha blivit mördad i samband me...
2,05 mars 2020 kl. 16.44,Misstänkt mord i Göteborg,Ett misstänkt mord utreds sedan en död person ...
3,17 september 2020 kl. 12.47,Man gripen för mord vid Heden i Göteborg,En man i 40-årsåldern har gripits som misstänk...
4,26 juni 2020 kl. 09.58,Åtalas för mord på kvinna i Göteborg,En man i 35-årsåldern åtalas vid Göteborgs De...


In [22]:
df2.to_csv("data_mord_aftonbladet.csv", sep=",")

In [33]:
day = [n.split(" ")[1] if n != "NULL" else n for n in dates ]
month = [n.split(" ")[2] if n != "NULL" else n for n in dates ]
year = [n.split(" ")[3] if n != "NULL" else n for n in dates ]

In [40]:
df2["day"] = day
df2["month"] = month
df2["year"] = year
df2["link"] = df["link"]

In [41]:
df2.head()

,date,title,text,day,month,year,link
0,28 maj 2020 kl. 12.07,Anhållen efter misstänkt mord i Göteborg,Polisen misstänker att en man kan ha blivit mö...,28,maj,2020,https://www.aftonbladet.se/nyheter/a/GGkbjq/an...
1,07 januari 2021 kl. 08.35,Misstänkt mord i Göteborg,En man misstänks ha blivit mördad i samband me...,07,januari,2021,https://www.aftonbladet.se/nyheter/a/Qm5XXx/mi...
2,05 mars 2020 kl. 16.44,Misstänkt mord i Göteborg,Ett misstänkt mord utreds sedan en död person ...,05,mars,2020,https://www.aftonbladet.se/nyheter/a/Joyb46/mi...
3,17 september 2020 kl. 12.47,Man gripen för mord vid Heden i Göteborg,En man i 40-årsåldern har gripits som misstänk...,17,september,2020,https://www.aftonbladet.se/nyheter/a/Al20Jx/ma...
4,26 juni 2020 kl. 09.58,Åtalas för mord på kvinna i Göteborg,En man i 35-årsåldern åtalas vid Göteborgs De...,26,juni,2020,https://www.aftonbladet.se/nyheter/a/rAkno3/at...


In [42]:
df2.to_csv("data_mord_aftonbladet.csv", sep=",")